# DSCI552: HW5



***Name: Cheng Peng***


***USC ID: 6898-9638-37***

In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import hamming_loss, silhouette_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

# Part I: Multi-class and Multi-label Classification using SVM 

# A): Download Data

In [ ]:
url = "/content/drive/My Drive/DSCI552/Homework 5 Data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv"
df = pd.read_csv(url)

In [ ]:
df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,MFCCs_11,MFCCs_12,MFCCs_13,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,0.188654,-0.075622,-0.156436,0.082245,0.135752,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,0.270958,-0.095004,-0.254341,0.022786,0.163320,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,0.266064,-0.072827,-0.237384,0.050791,0.207338,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,0.267279,-0.162258,-0.317084,-0.011567,0.100413,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,0.332695,-0.100749,-0.298524,0.037439,0.219153,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,0.065905,-0.000717,-0.145668,-0.059364,0.024206,-0.000861,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,0.038545,0.000856,-0.164675,-0.105600,0.030767,0.006457,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,0.062748,0.011868,-0.150025,-0.078615,0.024861,0.008696,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,0.049792,-0.003997,-0.153120,-0.075320,0.022903,0.001924,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [ ]:
train, test = train_test_split(df, test_size = 0.3, random_state = 1)

In [ ]:
df["Family"].value_counts()

Leptodactylidae    4420
Hylidae            2165
Dendrobatidae       542
Bufonidae            68
Name: Family, dtype: int64

In [ ]:
df["Genus"].value_counts()

Adenomera        4150
Hypsiboas        1593
Ameerega          542
Dendropsophus     310
Leptodactylus     270
Scinax            148
Osteocephalus     114
Rhinella           68
Name: Genus, dtype: int64

In [ ]:
df["Species"].value_counts()

AdenomeraHylaedactylus    3478
HypsiboasCordobae         1121
AdenomeraAndre             672
Ameeregatrivittata         542
HypsiboasCinerascens       472
HylaMinuta                 310
LeptodactylusFuscus        270
ScinaxRuber                148
OsteocephalusOophagus      114
Rhinellagranulosa           68
Name: Species, dtype: int64

In [ ]:
train_x, train_y_f, train_y_g, train_y_s = train.iloc[:,:-4], train["Family"], train["Genus"], train["Species"]
test_x, test_y_f, test_y_g, test_y_s = test.iloc[:,:-4], test["Family"], test["Genus"], test["Species"]

# B): Classification 

i): Exact Match and Hamming Score


*   exact match: the sample is correctly classified over all the labels 
*   hamming score: number of exact match samples over total number of samples 
*   hamming loss: 1 - hamming score



ii): SVM 

In [ ]:
# find the very large and very small parameters 
result_c = []
c_values = np.linspace(-10, 10, 21)
for c in c_values:
    model = SVC(C = 10**c).fit(train_x, train_y_f)
    score = model.score(test_x, test_y_f)
    result_c.append((c, score))
result_c

[(-10.0, 0.6150995831403427),
 (-9.0, 0.6150995831403427),
 (-8.0, 0.6150995831403427),
 (-7.0, 0.6150995831403427),
 (-6.0, 0.6150995831403427),
 (-5.0, 0.6150995831403427),
 (-4.0, 0.6150995831403427),
 (-3.0, 0.6150995831403427),
 (-2.0, 0.8517832329782307),
 (-1.0, 0.952755905511811),
 (0.0, 0.9773043075497916),
 (1.0, 0.9874942102825383),
 (2.0, 0.9902732746641963),
 (3.0, 0.9888837424733673),
 (4.0, 0.9888837424733673),
 (5.0, 0.9888837424733673),
 (6.0, 0.9888837424733673),
 (7.0, 0.9888837424733673),
 (8.0, 0.9888837424733673),
 (9.0, 0.9888837424733673),
 (10.0, 0.9888837424733673)]

Choose C between 0 and 3.0 in logscale, if we set the threshold to 0.97. Note the accuracy stays at the same level after 3.0

Choose C from 1 to 9 in log scale, if we set the threshodl at 0.98

In [ ]:
gammas = np.linspace(0.1, 3, 30)
result_gamma = []
for gamma in gammas:
    model = SVC(gamma = gamma).fit(train_x, train_y_f)
    score = model.score(test_x, test_y_f)
    result_gamma.append((gamma, score))

In [ ]:
result_gamma

[(0.1, 0.9458082445576655),
 (0.2, 0.9569245020842982),
 (0.3, 0.9647985178323297),
 (0.4, 0.9666512274201019),
 (0.5, 0.9749884205650764),
 (0.6, 0.9773043075497916),
 (0.7, 0.9791570171375636),
 (0.7999999999999999, 0.9805465493283928),
 (0.8999999999999999, 0.9810097267253358),
 (0.9999999999999999, 0.9819360815192218),
 (1.0999999999999999, 0.9828624363131079),
 (1.2, 0.9828624363131079),
 (1.3, 0.984251968503937),
 (1.4, 0.983788791106994),
 (1.5, 0.984251968503937),
 (1.5999999999999999, 0.9856415006947661),
 (1.7, 0.9861046780917091),
 (1.8, 0.9861046780917091),
 (1.9, 0.9870310328855951),
 (2.0, 0.9874942102825383),
 (2.0999999999999996, 0.9888837424733673),
 (2.1999999999999997, 0.9888837424733673),
 (2.3, 0.9888837424733673),
 (2.4, 0.9888837424733673),
 (2.5, 0.9888837424733673),
 (2.6, 0.9888837424733673),
 (2.6999999999999997, 0.9893469198703103),
 (2.8, 0.9898100972672533),
 (2.9, 0.9898100972672533),
 (3.0, 0.9898100972672533)]

Choose gamma from 0.8 to 2.0, if we set the threshold at 0.98. 

---



In [ ]:
grid = {"C": np.logspace(0, 3, 10), "gamma":np.linspace(0.8, 2.0, 10)}
model = SVC(kernel='rbf')
# search through grid for family 
grid_search_f = GridSearchCV(model, param_grid = grid, cv = 10, verbose = 1).fit(train_x, train_y_f)

# search through grid for Genus	
grid_search_g = GridSearchCV(model, param_grid = grid, cv = 10, verbose = 1).fit(train_x, train_y_g)

# search through grid for species 
grid_search_s = GridSearchCV(model, param_grid = grid, cv = 10, verbose = 1).fit(train_x, train_y_s)



Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  2.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  3.3min finished


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  3.0min finished


In [ ]:
c_f, gamma_f = grid_search_f.best_params_["C"], grid_search_f.best_params_["gamma"]
c_g, gamma_g = grid_search_g.best_params_["C"], grid_search_g.best_params_["gamma"]
c_s, gamma_s = grid_search_s.best_params_["C"], grid_search_s.best_params_["gamma"]

In [ ]:
# rebuild the model to evaluate 
model_f = SVC(C=c_f, gamma = gamma_f).fit(train_x, train_y_f)
model_g = SVC(C=c_g, gamma = gamma_g).fit(train_x, train_y_g)
model_s = SVC(C=c_s, gamma = gamma_s).fit(train_x, train_y_s)

In [ ]:
predict_f = model_f.predict(test_x)
predict_g = model_g.predict(test_x)
predict_s = model_s.predict(test_x)
loss_f = hamming_loss(test_y_f, predict_f) 
loss_g = hamming_loss(test_y_g, predict_g)
loss_s = hamming_loss(test_y_s, predict_s)


In [ ]:
print("The optimal parameters for label family: C={:.4f}, gamma={:4f} \nTest set hamming score = {:4f}, hamming loss = {:4f} \n".format(c_f, gamma_f, 1-loss_f, loss_f))
print("The optimal parameters for label genus: C={:.4f}, gamma={:4f} \nTest set hamming score = {:4f}, hamming loss = {:4f} \n".format(c_g, gamma_g, 1-loss_g, loss_g))
print("The optimal parameters for label species: C={:.4f}, gamma={:4f} \nTest set hamming score = {:4f}, hamming loss = {:4f} \n".format(c_s, gamma_s, 1-loss_s, loss_s))



The optimal parameters for label family: C=46.4159, gamma=2.000000 
Test set hamming score = 0.991200, hamming loss = 0.008800 

The optimal parameters for label genus: C=46.4159, gamma=1.866667 
Test set hamming score = 0.988421, hamming loss = 0.011579 

The optimal parameters for label species: C=46.4159, gamma=1.866667 
Test set hamming score = 0.989347, hamming loss = 0.010653 



iii): L1 Pentalized SVM

In [ ]:
# standardize the data 
scaler = StandardScaler().fit(train_x)
train_x_stand = pd.DataFrame(scaler.transform(train_x), columns = train_x.columns)
test_x_stand = pd.DataFrame(scaler.transform(test_x), columns = test_x.columns)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model = LinearSVC(penalty = 'l1', dual = False, random_state=1)
grid = {"C": np.logspace(0, 3, 10)}
# search through grid for family 
grid_search_f = GridSearchCV(model, param_grid = grid, cv = 10, verbose = 1).fit(train_x_stand, train_y_f)
# search through grid for Genus	
grid_search_g = GridSearchCV(model, param_grid = grid, cv = 10, verbose = 1).fit(train_x_stand, train_y_g)
# search through grid for species 
grid_search_s = GridSearchCV(model, param_grid = grid, cv = 10, verbose = 1).fit(train_x_stand, train_y_s)


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.2min finished


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.8min finished


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.9min finished


In [ ]:
c_f = grid_search_f.best_params_["C"]
c_g = grid_search_g.best_params_["C"]
c_s = grid_search_s.best_params_["C"]
# rebuild the model to evaluate 
model_f = LinearSVC(C=c_f, penalty = 'l1', dual = False, random_state=1).fit(train_x_stand, train_y_f)
model_g = LinearSVC(C=c_g, penalty = 'l1', dual = False, random_state=1).fit(train_x_stand, train_y_g)
model_s = LinearSVC(C=c_s, penalty = 'l1', dual = False, random_state=1).fit(train_x_stand, train_y_s)
predict_f = model_f.predict(test_x_stand)
predict_g = model_g.predict(test_x_stand)
predict_s = model_s.predict(test_x_stand)
loss_f = hamming_loss(test_y_f, predict_f) 
loss_g = hamming_loss(test_y_g, predict_g)
loss_s = hamming_loss(test_y_s, predict_s)
print("The optimal parameters for label family: C={:.4f}\nTest set hamming score = {:4f}, hamming loss = {:4f} \n".format(c_f, 1-loss_f, loss_f))
print("The optimal parameters for label genus: C={:.4f}\nTest set hamming score = {:4f}, hamming loss = {:4f} \n".format(c_g, 1-loss_g, loss_g))
print("The optimal parameters for label species: C={:.4f}\nTest set hamming score = {:4f}, hamming loss = {:4f} \n".format(c_s, 1-loss_s, loss_s))


The optimal parameters for label family: C=1.0000
Test set hamming score = 0.933766, hamming loss = 0.066234 

The optimal parameters for label genus: C=21.5443
Test set hamming score = 0.947198, hamming loss = 0.052802 

The optimal parameters for label species: C=2.1544
Test set hamming score = 0.956461, hamming loss = 0.043539 



iv): Imbalance Class

In [ ]:
smote = SMOTE(random_state=1)
train_x_f_smote, train_y_f_smote = smote.fit_resample(train_x_stand, train_y_f)
train_x_g_smote, train_y_g_smote = smote.fit_resample(train_x_stand, train_y_g)
train_x_s_smote, train_y_s_smote = smote.fit_resample(train_x_stand, train_y_s)

In [ ]:
train_x_f_smote = pd.DataFrame(train_x_f_smote, columns = train_x.columns)
train_x_g_smote = pd.DataFrame(train_x_g_smote, columns = train_x.columns)
train_x_s_smote = pd.DataFrame(train_x_s_smote, columns = train_x.columns)
train_y_f_smote = pd.DataFrame(train_y_f_smote)
train_y_g_smote = pd.DataFrame(train_y_g_smote)
train_y_s_smote = pd.DataFrame(train_y_s_smote)


In [ ]:
print("There are {} samples for lable family\n".format(len(train_y_f_smote)))
print(train_y_f_smote.value_counts())
print("There are {} samples for lable genus\n".format(len(train_y_g_smote)))
print(train_y_g_smote.value_counts())
print("There are {} samples for lable species\n".format(len(train_y_s_smote)))
print(train_y_s_smote.value_counts())

There are 12368 samples for lable family

Leptodactylidae    3092
Hylidae            3092
Dendrobatidae      3092
Bufonidae          3092
dtype: int64
There are 23128 samples for lable genus

Scinax           2891
Rhinella         2891
Osteocephalus    2891
Leptodactylus    2891
Hypsiboas        2891
Dendropsophus    2891
Ameerega         2891
Adenomera        2891
dtype: int64
There are 24040 samples for lable species

ScinaxRuber               2404
Rhinellagranulosa         2404
OsteocephalusOophagus     2404
LeptodactylusFuscus       2404
HypsiboasCordobae         2404
HypsiboasCinerascens      2404
HylaMinuta                2404
Ameeregatrivittata        2404
AdenomeraHylaedactylus    2404
AdenomeraAndre            2404
dtype: int64


In [ ]:
model = LinearSVC(penalty = 'l1', dual = False, random_state=1)
grid = {"C": np.logspace(0, 3, 10)}
# search through grid for family 
grid_search_f = GridSearchCV(model, param_grid = grid, cv = 10, verbose = 1).fit(train_x_f_smote, train_y_f_smote)
# search through grid for Genus	
grid_search_g = GridSearchCV(model, param_grid = grid, cv = 10, verbose = 1).fit(train_x_g_smote, train_y_g_smote)
# search through grid for species 
grid_search_s = GridSearchCV(model, param_grid = grid, cv = 10, verbose = 1).fit(train_x_s_smote, train_y_s_smote)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 13.1min finished


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 37.0min finished


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 42.6min finished


In [ ]:
c_f = grid_search_f.best_params_["C"]
c_g = grid_search_g.best_params_["C"]
c_s = grid_search_s.best_params_["C"]
# rebuild the model to evaluate 
model_f = LinearSVC(C=c_f, penalty = 'l1', dual = False, random_state=1).fit(train_x_f_smote, train_y_f_smote)
model_g = LinearSVC(C=c_g, penalty = 'l1', dual = False, random_state=1).fit(train_x_g_smote, train_y_g_smote)
model_s = LinearSVC(C=c_s, penalty = 'l1', dual = False, random_state=1).fit(train_x_s_smote, train_y_s_smote)
predict_f = model_f.predict(test_x_stand)
predict_g = model_g.predict(test_x_stand)
predict_s = model_s.predict(test_x_stand)
loss_f = hamming_loss(test_y_f, predict_f) 
loss_g = hamming_loss(test_y_g, predict_g)
loss_s = hamming_loss(test_y_s, predict_s)
print("The optimal parameters for label family: C={:.4f}\nTest set hamming score = {:4f}, hamming loss = {:4f} \n".format(c_f, 1-loss_f, loss_f))
print("The optimal parameters for label genus: C={:.4f}\nTest set hamming score = {:4f}, hamming loss = {:4f} \n".format(c_g, 1-loss_g, loss_g))
print("The optimal parameters for label species: C={:.4f}\nTest set hamming score = {:4f}, hamming loss = {:4f} \n".format(c_s, 1-loss_s, loss_s))


The optimal parameters for label family: C=4.6416
Test set hamming score = 0.926818, hamming loss = 0.073182 

The optimal parameters for label genus: C=46.4159
Test set hamming score = 0.926355, hamming loss = 0.073645 

The optimal parameters for label species: C=10.0000
Test set hamming score = 0.953219, hamming loss = 0.046781 



v): Classifier Chian Method 

vi): Multi-label Classification Metrics

# Part 2: K-Mean Clustering on a Multi-class and Multi-label Dataset 

***Terminologies:***


*   hamming score: the number of correctly labeled samples over the total number of samples
*   hamming loss: 1-hamming score 
*   hamming distance: the number of mis-labeled samples, hamming distance = hamming loss * total number of samples 



In [ ]:
# use the entrie dataset as the training data 
train_x = df.iloc[:,:-4]
train_y = df.iloc[:,-4:-1]
train_y_f = df["Family"]
train_y_g = df["Genus"]
train_y_s = df["Species"]

In [ ]:
# use the Silhouettes score provided by sklearn 
# Monte Carlo simulation, run kmean 50 times 
def monteCarlo():
    optimal_k = -1
    optimal_score = np.inf *(-1)
    optimal_model = []

    for k in range(2, 52):
        model = KMeans(n_clusters=k).fit(train_x)
        labels = model.labels_
        score = silhouette_score(train_x, labels)
        if score > optimal_score:
            optimal_k = k
            optimal_score = score
            optimal_model = model

    print("The optimal K={}, and the Silhouettes score = {:.4f}".format(optimal_k, optimal_score))

    # use the optimal model to carry out the classification 
    predict = optimal_model.labels_
    predict = pd.DataFrame(predict, columns = ["Cluster"])
    #find the index of samples belonging to each clusters 
    cluster_index = {}
    for i in range(optimal_k):
        cluster_index[i] = predict[predict["Cluster"]==i].index
        # majority of labels of each class in clusters
    majority = {} 
    for i in range(optimal_k):
        majority_f = train_y_f.iloc[cluster_index[i]].value_counts().index[0]
        majority_g = train_y_g.iloc[cluster_index[i]].value_counts().index[0]
        majority_s = train_y_s.iloc[cluster_index[i]].value_counts().index[0]
        majority[i] = (majority_f, majority_g, majority_s)
        print("For cluster {}. the majority labels are: {}, {}, {}".format(i, majority_f, majority_g, majority_s))
    
    #assign the samples in the cluster to the majority label 
    predict_f = [majority[x][0] for x in predict["Cluster"]]
    predict_g = [majority[x][1] for x in predict["Cluster"]]
    predict_s = [majority[x][2] for x in predict["Cluster"]]
    hamming_loss_f = hamming_loss(train_y_f, predict_f)
    hamming_loss_g = hamming_loss(train_y_g, predict_g)
    hamming_loss_s = hamming_loss(train_y_s, predict_s)
    hamming_loss_value = np.mean([hamming_loss_f, hamming_loss_g, hamming_loss_s])
    hamming_score = 1-hamming_loss_value
    hamming_distance = hamming_loss_value * len(train_y_f)*3
    print("Hamming loss = {:.4f}, hamming score = {:.4f}, hamming distance = {:.4f}".format(hamming_loss_value, hamming_score, hamming_distance))
    return hamming_loss_value, hamming_score, hamming_distance

***Comment:*** I used K from 2 to 51 as opposed to 1 to 50 as stated on the handout since silhouette score reuiqres more than 1 cluster to be calculated. In addition, K=1 does not really make sense as we just put all samples into one cluster. 

In [ ]:
table = []
for i in range(50):
    print("Iteration {}".format(i))
    h1, h2, h3 = monteCarlo()
    table.append([h1, h2, h3])
    print("\n")

Iteration 0
The optimal K=4, and the Silhouettes score = 0.3786
For cluster 0. the majority labels are: Hylidae, Hypsiboas, HypsiboasCinerascens
For cluster 1. the majority labels are: Leptodactylidae, Adenomera, AdenomeraHylaedactylus
For cluster 2. the majority labels are: Dendrobatidae, Ameerega, Ameeregatrivittata
For cluster 3. the majority labels are: Hylidae, Hypsiboas, HypsiboasCordobae
Hamming loss = 0.2225, hamming score = 0.7775, hamming distance = 4802.0000


Iteration 1
The optimal K=4, and the Silhouettes score = 0.3839
For cluster 0. the majority labels are: Leptodactylidae, Adenomera, AdenomeraHylaedactylus
For cluster 1. the majority labels are: Hylidae, Hypsiboas, HypsiboasCordobae
For cluster 2. the majority labels are: Leptodactylidae, Adenomera, AdenomeraAndre
For cluster 3. the majority labels are: Hylidae, Hypsiboas, HypsiboasCinerascens
Hamming loss = 0.2341, hamming score = 0.7659, hamming distance = 5052.0000


Iteration 2
The optimal K=4, and the Silhouettes 

In [ ]:
table_df = pd.DataFrame(table, columns = ["Hamming Loss", "Hamming Score", "Hamming Distance"])

In [ ]:
table_df.head()

,Hamming Loss,Hamming Score,Hamming Distance
0,0.222469,0.777531,4802.0
1,0.234051,0.765949,5052.0
2,0.222423,0.777577,4801.0
3,0.222423,0.777577,4801.0
4,0.222423,0.777577,4801.0


In [ ]:
table_df.agg(["mean","std"])

,Hamming Loss,Hamming Score,Hamming Distance
mean,0.223997,0.776003,4834.980000
std,0.004646,0.004646,100.280523


***Comment:*** 




* The hamming distance is the sum of all mis-labeled samples over all the labels in family, genus and species.
* Hamming distance = number of all mis-labeled samples / (3 * number of samples )


